In [ ]:
%load_ext pystackql.magics

In [ ]:
regions = "'ap-southeast-2', 'us-east-1'"
region = 'ap-southeast-2'

In [ ]:
%%stackql
SELECT 
state, 
count(*) as num_instances
FROM aws.ec2.instances 
WHERE region IN ($regions)
GROUP BY state

In [ ]:
_.plot(kind='bar', title='Instances by State', x='state', y='num_instances')

In [ ]:
%%stackql
SELECT region, instanceType, COUNT(*) as num_instances
FROM aws.ec2.instances
WHERE region IN ($regions)
GROUP BY instanceType

In [ ]:
_.plot(kind='pie', y='num_instances', labels=_['instanceType'], title='Instances by Type', autopct='%1.1f%%')

In [ ]:
%%stackql
CREATE MATERIALIZED VIEW vw_ec2_instance_types AS
SELECT 
memoryInfo,
hypervisor,
autoRecoverySupported,
instanceType,
SPLIT_PART(processorInfo, '\n', 3) as processorArch,
currentGeneration,
freeTierEligible,
hibernationSupported,
SPLIT_PART(vCpuInfo, '\n', 2) as vCPUs,
bareMetal,
burstablePerformanceSupported,
dedicatedHostsSupported
FROM aws.ec2.instance_types
WHERE region = '$region'

In [ ]:
%%stackql
select PolicyName, Arn from aws.iam.policies where region = '$region'

In [ ]:
%%stackql
SELECT region, function_name
FROM aws.lambda.functions
WHERE region IN (
'us-east-1',
'us-east-2',
'us-west-1',
'us-west-2',
'ap-south-1',
'ap-northeast-3',
'ap-northeast-2',
'ap-southeast-1',
'ap-southeast-2',
'ap-northeast-1',
'ca-central-1',
'eu-central-1',
'eu-west-1',
'eu-west-2',
'eu-west-3',
'eu-north-1',
'sa-east-1')

In [ ]:
functions_list = _.to_dict(orient='records')

In [ ]:
from pystackql import StackQL
stackql = StackQL(download_dir='/srv/stackql', app_root='/jupyter/.stackql', output='pandas')
stackql.executeStmt("REGISTRY PULL aws")

In [ ]:
# get all functions across all regions asynchronously using pystackql
import nest_asyncio
nest_asyncio.apply()
get_fns_details = [
    f'''
    SELECT 
    function_name,
    region,
    arn,
    description,
    architectures,
    memory_size,
    runtime
    FROM aws.lambda.function
    WHERE region = '{function['region']}'
    AND data__Identifier = '{function['function_name']}'
    '''
    for function in functions_list
]
function_details_df = await stackql.executeQueriesAsync(get_fns_details)
function_details_df